# Text Classification Notebook

In [2]:
!pip install --upgrade pip
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install PySastrawi
!pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Machine Learning related imports
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input

In [4]:
# Dataset and generic imports
import pandas as pd
import numpy as np
import sklearn
import os
import string

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [5]:
# Load dataset
filepath = os.getcwd()
datasetpath =  os.path.join(filepath, "cleaned_data", "laporanencoded.csv")

# Github URL for dataset when used in Google Colab
github_url = "https://raw.githubusercontent.com/Capstone-Project-B21-CAP0113/ml-tf/main/laporan/cleaned_data/laporanencoded.csv"

laporan = pd.read_csv(datasetpath, encoding="ISO-8859-1")
# Print dataset shape
print(laporan.shape)
# Print dataset head
laporan.head()


(2852, 27)


,text,perselisihan,infrastruktur,pemerintah,kesehatan,teknologi,administrasi,fasilitas,lingkungan,ketertiban,...,air,pendidikan,kebersihan,sosial,wisata,sara,pencurian,korupsi,bbm,keuangan
0,mohon bantuannya untuk menormalkan sistem di l...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ass pak gubsaya perangkat desa karangsari kec ...,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,selamat sore bapak gubernur atau yang mewakili...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,assalamualaikum pak gub saya pengurus paguyuba...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,lapor saya okada arle sandi email okadaarlegm...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Split text and labels
label_list = [
    "perselisihan",
    "infrastruktur",
    "pemerintah",
    "kesehatan",
    "teknologi",
    "administrasi",
    "fasilitas",
    "lingkungan",
    "ketertiban",
    "listrik",
    "bahaya",
    "lainnya",
    "pungli",
    "ilegal",
    "lalulintas",
    "bencana",
    "air",
    "pendidikan",
    "kebersihan",
    "sosial",
    "wisata",
    "sara",
    "pencurian",
    "korupsi",
    "bbm",
    "keuangan"
] 
x = laporan["text"]
y = laporan[label_list]

# Print number of element in each category ( one element can have many label since it's a multi label classification )
for i in label_list:
    print("{}: {}".format(i, (laporan[i] == 1).sum()))

perselisihan: 16
infrastruktur: 353
pemerintah: 68
kesehatan: 80
teknologi: 23
administrasi: 693
fasilitas: 73
lingkungan: 114
ketertiban: 87
listrik: 43
bahaya: 22
lainnya: 139
pungli: 106
ilegal: 47
lalulintas: 42
bencana: 58
air: 85
pendidikan: 134
kebersihan: 9
sosial: 181
wisata: 13
sara: 7
pencurian: 6
korupsi: 39
bbm: 1
keuangan: 82


In [7]:
# Text head
x.head()

0    mohon bantuannya untuk menormalkan sistem di l...
1    ass pak gubsaya perangkat desa karangsari kec ...
2    selamat sore bapak gubernur atau yang mewakili...
3    assalamualaikum pak gub saya pengurus paguyuba...
4    lapor saya okada arle sandi email  okadaarlegm...
Name: text, dtype: object

In [8]:
# Label head
y.head()

,perselisihan,infrastruktur,pemerintah,kesehatan,teknologi,administrasi,fasilitas,lingkungan,ketertiban,listrik,...,air,pendidikan,kebersihan,sosial,wisata,sara,pencurian,korupsi,bbm,keuangan
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Remove stopwords and stem words using Sastrawi
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stopword_factory = StopWordRemoverFactory()
stopword = stopword_factory.create_stop_word_remover()

for i in range(len(x)):
    x[i] = stopword.remove(x[i])
    x[i] = stemmer.stem(x[i])

x.head()

<ipython-input-9-7f4537c43ddc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = stopword.remove(x[i])
<ipython-input-9-7f4537c43ddc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = stemmer.stem(x[i])


0    bantu normal sistem lpse kabupaten demak mana ...
1    ass gubsaya perangkat desa karangsari kec kara...
2    selamat sore gubernur mewakilisaya ahmad rofiq...
3    assalamualaikum gub urus paguyuban dagang puja...
4    lapor okada arle sandi email okadaarlegmailcom...
Name: text, dtype: object

In [10]:
# Tokenize and pad text
vocab_size = 2000
embedding_dim = 16
max_length = 300
trunc_type = "post"
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x)

sequences = tokenizer.texts_to_sequences(x)

padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

In [11]:
# Shuffle, batch and separate data into train, dev and test
BUFFER_SIZE = 1000
BATCH_SIZE = 32
DATASET_SIZE = len(x)


tx = tf.convert_to_tensor(padded)
ty = tf.convert_to_tensor(y)

dataset = tf.data.Dataset.from_tensor_slices((tx, ty))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

train_set = dataset.take(int(0.8 * len(dataset)))
test_set = dataset.skip(int(0.8 * len(dataset)))
test_set = dataset.take(int(0.2 * len(dataset))) 

print(len(train_set))
print(len(test_set))

72
18


In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(26, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         256000    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         263168    
_________________________________________________________________
dropout (Dropout)            (None, None, 256)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               164352    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
NUM_EPOCHS = 75
history = model.fit(train_set, epochs=NUM_EPOCHS, validation_data=test_set)

Epoch 1/75
72/72 [==============================] - 81s 993ms/step - loss: 0.3038 - accuracy: 0.1354 - val_loss: 0.1246 - val_accuracy: 0.2205
Epoch 2/75
72/72 [==============================] - 73s 1s/step - loss: 0.1334 - accuracy: 0.2274 - val_loss: 0.1178 - val_accuracy: 0.2326
Epoch 3/75
72/72 [==============================] - 75s 1s/step - loss: 0.1318 - accuracy: 0.2344 - val_loss: 0.1198 - val_accuracy: 0.2309
Epoch 4/75
72/72 [==============================] - 75s 1s/step - loss: 0.1298 - accuracy: 0.2400 - val_loss: 0.1158 - val_accuracy: 0.2361
Epoch 5/75
72/72 [==============================] - 75s 1s/step - loss: 0.1283 - accuracy: 0.2378 - val_loss: 0.1160 - val_accuracy: 0.2535
Epoch 6/75
72/72 [==============================] - 76s 1s/step - loss: 0.1281 - accuracy: 0.2378 - val_loss: 0.1168 - val_accuracy: 0.2569
Epoch 7/75
72/72 [==============================] - 77s 1s/step - loss: 0.1282 - accuracy: 0.2361 - val_loss: 0.1168 - val_accuracy: 0.2274
Epoch 8/75
72/72 

In [15]:
# Text cleaning function for inferrence
def treatinput(inp):
    # Make string into lowercase string
    treated = inp.lower()
    # remove punctuation
    treated = treated.translate(str.maketrans("","",string.punctuation))
    # remove trailing whitespace
    treated = treated.strip()
    # Remove stopwords
    treated = stopword.remove(treated)
    # Stem string
    treated = stemmer.stem(treated)
    return treated

In [16]:
# sample inferrece
sample_laporan = "Jalan sukamaju jaya macet sudah dari pagi, lampu merah mati dan tidak ada polisi lalu lintas"
treated_input = treatinput(sample_laporan)
print(treated_input)
pad = pad_sequences(tokenizer.texts_to_sequences([treated_input]), maxlen=max_length, truncating=trunc_type)
prediction = model.predict(pad)

res = dict(zip(label_list, prediction[0]))

jalan sukamaju jaya macet pagi lampu merah mati polisi lintas


In [17]:
# Print predictions
for i in res.keys():
    # if(res[i] > 0.6):
    print("{} {:.5f}".format(i, res[i]))

perselisihan 0.00000
infrastruktur 0.00334
pemerintah 0.00003
kesehatan 0.00000
teknologi 0.00000
administrasi 0.00000
fasilitas 0.02625
lingkungan 0.00254
ketertiban 0.00001
listrik 0.00081
bahaya 0.06766
lainnya 0.00016
pungli 0.00000
ilegal 0.00219
lalulintas 0.66482
bencana 0.15976
air 0.00004
pendidikan 0.00000
kebersihan 0.01268
sosial 0.00005
wisata 0.00000
sara 0.00394
pencurian 0.00000
korupsi 0.00000
bbm 0.00000
keuangan 0.00000


In [18]:
# Try a new model with Keras Tuner
import kerastuner as kt

In [19]:
# Define Model
def model_builder(hp):
    emb_units = hp.Int('emb_units', min_value=16, max_value=256, step=16)
    bid1_units = hp.Int('bid1_units', min_value=16, max_value=256, step=16)
    drop1_frac = hp.Float('drop1_frac', min_value=0.1, max_value=0.8, step=0.1)
    bid2_units = hp.Int('bid2_units', min_value=16, max_value=256, step=16)
    drop2_frac = hp.Float('drop2_frac', min_value=0.1, max_value=0.8, step=0.1)
    dense1_units = hp.Int('dense1_unit', min_value=16, max_value=256, step=16)
    drop3_frac = hp.Float('drop3_frac', min_value=0.1, max_value=0.8, step=0.1)
    dense2_units = hp.Int('dense2_unit', min_value=16, max_value=256, step=16)

    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, emb_units),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(bid1_units, return_sequences=True)),
    tf.keras.layers.Dropout(drop1_frac),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(bid2_units)),
    tf.keras.layers.Dropout(drop2_frac),
    tf.keras.layers.Dense(dense1_units, activation='relu'),
    tf.keras.layers.Dropout(drop3_frac),
    tf.keras.layers.Dense(dense2_units, activation='relu'),
    tf.keras.layers.Dense(26, activation='sigmoid')
])

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), metrics=['accuracy'])
    
    return model

In [22]:
tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=100, factor=3, overwrite=True)

In [23]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [24]:
tuner.search(train_set, epochs=100, validation_data=test_set, callbacks=[stop_early])

best_hps= tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps)

Trial 107 Complete [00h 23m 40s]
val_accuracy: 0.2309027761220932

Best val_accuracy So Far: 0.3333333432674408
Total elapsed time: 18h 00m 29s

Search: Running Trial #108

Hyperparameter    |Value             |Best Value So Far 
emb_units         |64                |48                
bid1_units        |96                |80                
drop1_frac        |0.5               |0.4               
bid2_units        |192               |48                
drop2_frac        |0.2               |0.1               
dense1_unit       |128               |208               
drop3_frac        |0.7               |0.3               
dense2_unit       |176               |240               
learning_rate     |0.01              |0.01              
tuner/epochs      |4                 |2                 
tuner/initial_e...|2                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |1                 |0                 
tuner/trial_id    |111d4ca2ad

KeyboardInterrupt: 

In [ ]:
# Trial 5 Complete [00h 03m 30s]
# val_accuracy: 0.2395833283662796

# Best val_accuracy So Far: 0.2534722089767456
# Total elapsed time: 00h 55m 20s

# Search: Running Trial #6

# Hyperparameter    |Value             |Best Value So Far 
# emb_units         |48                |32                
# bid1_units        |240               |112               
# drop1_frac        |0.6               |0.3               
# bid2_units        |176               |112               
# drop2_frac        |0.4               |0.4               
# dense1_unit       |176               |112               
# drop3_frac        |0.7               |0.4               
# dense2_unit       |144               |224               
# learning_rate     |1e-05             |0.01              
# tuner/epochs      |2                 |2                 
# tuner/initial_e...|0                 |0                 
# tuner/bracket     |2                 |2                 
# tuner/round       |0                 |0                 

# Epoch 1/2
# 72/72 [==============================] - 618s 8s/step - loss: 0.6918 - accuracy: 0.0573 - val_loss: 0.6889 - val_accuracy: 0.0625
# Epoch 2/2
# 72/72 [==============================] - 623s 9s/step - loss: 0.6846 - accuracy: 0.0760 - val_loss: 0.6749 - val_accuracy: 0.0521

In [1]:
model_t = tuner.hypermodel.build(best_hps)
history = model_t.fit(dataset, epochs=100, validation_data=test_set)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

NameError: name 'tuner' is not defined